In [1]:
import utils 
import csv 
import pandas as pd

text_files, json_files = utils.return_file_path()
df = pd.DataFrame(columns=["Qustion", "Output", "Predict", "Answer", "Difficulty" ,"Number"])

for text_file, json_file in zip(text_files, json_files):
    output = "" 
    start_idx = 0 
    
    answer = ("prime" if "prime" in text_file else "composite")
    data_type = json_file.rstrip(".json").lstrip("../json_sampled/")
    querys = utils.return_json_data(json_file) 
    
    for query in querys:
        query_list = list(query.values())
        for i in range(len(query_list)-2):
            output, start_idx = utils.return_output(text_file, start_idx)
            if i >= 4:
                continue
            result = utils.return_PrimeOrComposite(output, i)
            
            factor1 = query["question5"][query["question5"].find("?")]
            
            # print(query["number"], f"Q{i+1}", output, result, query["answer"])
            df.loc[str(query["number"])+f"_Q{i+1}"] = (f"Q{i+1}", output, result, query["answer"], data_type ,query["number"], )
df

,Qustion,Output,Predict,Answer,Difficulty,Number
10_Q1,Q1,"\nNo, 10 is not a prime number. It can be divi...",composite,composite,2_digit_composite,10
10_Q2,Q2,"\nNo, 10 is not a prime number because it is d...",composite,composite,2_digit_composite,10
10_Q3,Q3,"\nNo, 10 is not a composite number. A composit...",prime,composite,2_digit_composite,10
10_Q4,Q4,"\nNo, 10 is not a composite number. A composit...",prime,composite,2_digit_composite,10
12_Q1,Q1,"\nNo, 12 is not a prime number. It is divisibl...",composite,composite,2_digit_composite,12
...,...,...,...,...,...,...
2939_Q4,Q4,"\nTo determine if 2939 is a composite number, ...",prime,prime,4_digit_prime,2939
6521_Q1,Q1,"\nYes, 6521 is a prime number.\n",prime,prime,4_digit_prime,6521
6521_Q2,Q2,"\nTo determine if 6521 is a prime number, we c...",composite,prime,4_digit_prime,6521
6521_Q3,Q3,"\nYes, 6521 is a composite number.\n",composite,prime,4_digit_prime,6521


In [2]:
df.groupby('Difficulty').count()


,Qustion,Output,Predict,Answer,Number
Difficulty,,,,,
2_digit_composite,276,276,276,276,276
2_digit_prime,84,84,84,84,84
3_digit_easy_composite,808,808,808,808,808
3_digit_medium_composite,224,224,224,224,224
3_digit_prime,572,572,572,572,572
4_digit_easy_composite,2000,2000,2000,2000,2000
4_digit_hard_composite,524,524,524,524,524
4_digit_medium_composite,2000,2000,2000,2000,2000
4_digit_prime,2000,2000,2000,2000,2000


In [4]:
import pandas as pd 
import numpy as np 

acc_df = pd.DataFrame(columns=["w/o CoT", "with CoT"])

undefined_df = pd.DataFrame(columns=["Qustion", "Output", "Predict", "Answer", "Difficulty" ,"Number"])

Difficulty_list = df['Difficulty'].unique()

right_composite = 0
wrong_composite = 0
for Difficulty in Difficulty_list:
    if Difficulty[-5:] == "prime":
        continue
    
    acc = np.array([0, 0, 0, 0])
    wrong_patton_acc = np.array([0, 0, 0, 0])
    
    temp_df = df[df['Difficulty'] == Difficulty]

    for idx, row in temp_df.iterrows():        
        i = int(idx[-1])-1
        if row['Predict'] == row['Answer']:
            acc[i] += 1
            if i==1 or i==3:
                right_composite += 1
        

1067 / 4244
633 / 1067
434 / 1067


,Q2,Q4
[wrong rate] 2_digit_composite,13.85,1.61
[wrong rate] 3_digit_easy_composite,9.03,7.26
[wrong rate] 3_digit_medium_composite,33.33,23.81
[wrong rate] 4_digit_easy_composite,14.98,9.13
[wrong rate] 4_digit_hard_composite,60.00,18.18
[wrong rate] 4_digit_medium_composite,55.56,24.56


In [13]:
# 2 ~ 10000 : prime number
n = 10000
a = [False, False] + [True] * (n-1)
primes = [] 

for i in range(2, n+1):
  if a[i]:
    primes.append(i)
    for j in range(2*i, n+1, i):
      a[j] = False 

In [25]:
pd.set_option('mode.chained_assignment',  None)

Difficulty_list = df['Difficulty'].unique()

answer_df = pd.DataFrame(columns=["Qustion", "Output", "Predict", "Answer", "Difficulty" ,"Number", "Factor1", "Factor2"])
composite_cnt = 0

for Difficulty in Difficulty_list:
    # skip prime number 
    if Difficulty[-5:] == "prime":
        continue
    
    temp_df = df[df['Difficulty'] == Difficulty]
    for idx, row in temp_df.iterrows():
        i = int(idx[-1])-1
        
        # Factors
        for prime in primes:
            if (row['Number']%prime) == 0:
                factor1 = prime 
                factor2 = row['Number']//prime
                break
        
        if row['Predict'] == row['Answer']:
            composite_cnt += 1
            answer_df.loc[f"{row['Number']}_Q{i+1}"] = row
            answer_df.loc[f"{row['Number']}_Q{i+1}", 'Factor1'] = factor1
            answer_df.loc[f"{row['Number']}_Q{i+1}", 'Factor2'] = factor2
    
    
print(composite_cnt, "/ 8488")
answer_df['Verdict'] = "correct"
print(len(answer_df))
answer_df.head()

3272 / 8488
3272


,Qustion,Output,Predict,Answer,Difficulty,Number,Factor1,Factor2,Verdict
10_Q1,Q1,"\nNo, 10 is not a prime number. It can be divi...",composite,composite,2_digit_composite,10,2,5,correct
10_Q2,Q2,"\nNo, 10 is not a prime number because it is d...",composite,composite,2_digit_composite,10,2,5,correct
12_Q1,Q1,"\nNo, 12 is not a prime number. It is divisibl...",composite,composite,2_digit_composite,12,2,6,correct
12_Q2,Q2,"\nTo determine if 12 is a prime number, we nee...",composite,composite,2_digit_composite,12,2,6,correct
12_Q4,Q4,"\nYes, 12 is a composite number. \n\nTo determ...",composite,composite,2_digit_composite,12,2,6,correct


In [27]:
# answer_df.to_csv("../csv/verdict_chatgpt.csv")

In [108]:
verdict_df = pd.read_csv("../csv/verdict_chatgpt.csv")

In [110]:
len(verdict_df)

3272

In [81]:
err_cnt = {}
for idx, row in verdict_df.iterrows():
    if row['Verdict'] == "wrong":
        try: 
            err_cnt[row['Difficulty']+"_"+row['Qustion']] += 1
        except: 
            err_cnt[row['Difficulty']+"_"+row['Qustion']] = 1

In [107]:
import pandas as pd 
import numpy as np 

acc_df = pd.DataFrame(columns=["Q1", "Q2", "Q3", "Q4"])
cnt_df = pd.DataFrame(columns=["Q1", "Q2", "Q3", "Q4"])

undefined_df = pd.DataFrame(columns=["Qustion", "Output", "Predict", "Answer", "Difficulty" ,"Number"])

Difficulty_list = df['Difficulty'].unique()

right_composite = 0
wrong_composite = 0

for Difficulty in Difficulty_list:
    if Difficulty[-5:] == "prime":
        continue
    
    acc = np.array([0, 0, 0, 0])
    cnt = np.array([0, 0, 0, 0])
    
    wrong_patton_acc = np.array([0, 0, 0, 0])
    
    temp_df = df[df['Difficulty'] == Difficulty]

    for idx, row in temp_df.iterrows():  
        i = int(idx[-1])-1
        cnt[i] += 1
        if row['Predict'] == row['Answer']:
            acc[i] += 1
        else: 
            continue
    acc_df.loc[Difficulty] = acc 
    cnt_df.loc[Difficulty] = acc 

err_df = pd.DataFrame(columns=["Q1", "Q2", "Q3", "Q4"])
for difficulty in verdict_df['Difficulty'].unique():
    for k, v in err_cnt.items():
        q = k[-2:]
        err_df.loc[k[:-3], q] = v
        
    
acc_df.round(2)

,Q1,Q2,Q3,Q4
2_digit_composite,69,65,47,62
3_digit_easy_composite,187,155,172,124
3_digit_medium_composite,50,18,53,21
4_digit_easy_composite,435,267,460,241
4_digit_hard_composite,55,10,107,11
4_digit_medium_composite,161,36,409,57


In [83]:
err_df = err_df.fillna(0)
err_df.sum().sum()

572

In [105]:
# Q1이 근거를 들 확률이 높음. 
# Q1, Q3를 보는게 맞느지 모르겠음. 
# 왜냐면 답만 말하라 했는데, 풀이까지 말했다가 틀리는 것이라. 
# 4_자루_쉬운_합성수는 현재 하는 중
(err_df / acc_df * 100).round(2).reindex(['2_digit_composite',
                                                   '3_digit_easy_composite', 
                                                   '4_digit_easy_composite',
                                                   '3_digit_medium_composite',
                                                   '4_digit_medium_composite', 
                                                   '4_digit_hard_composite'])

,Q1,Q2,Q3,Q4
2_digit_composite,NaN,4.615385,NaN,1.612903
3_digit_easy_composite,4.278075,14.193548,1.162791,10.483871
4_digit_easy_composite,27.586207,12.734082,4.782609,24.06639
3_digit_medium_composite,22.0,44.444444,9.433962,14.285714
4_digit_medium_composite,61.490683,66.666667,9.046455,78.947368
4_digit_hard_composite,54.545455,90.0,8.411215,81.818182


In [92]:
((acc_df-err_df) / cnt_df * 100).round(2).reindex(['2_digit_composite',
                                                   '3_digit_easy_composite', 
                                                   '4_digit_easy_composite',
                                                   '3_digit_medium_composite',
                                                   '4_digit_medium_composite', 
                                                   '4_digit_hard_composite']).to_csv("err_chatgpt.csv")

In [91]:
cnt_df

,Q1,Q2,Q3,Q4
2_digit_composite,69,69,69,69
3_digit_easy_composite,202,202,202,202
3_digit_medium_composite,56,56,56,56
4_digit_easy_composite,500,500,500,500
4_digit_hard_composite,131,131,131,131
4_digit_medium_composite,500,500,500,500


In [61]:
df.groupby('Difficulty').count()

,Qustion,Output,Predict,Answer,Number
Difficulty,,,,,
2_digit_composite,276,276,276,276,276
2_digit_prime,84,84,84,84,84
3_digit_easy_composite,808,808,808,808,808
3_digit_medium_composite,224,224,224,224,224
3_digit_prime,572,572,572,572,572
4_digit_easy_composite,2000,2000,2000,2000,2000
4_digit_hard_composite,524,524,524,524,524
4_digit_medium_composite,2000,2000,2000,2000,2000
4_digit_prime,2000,2000,2000,2000,2000


In [142]:
import numpy as np

acc_df = pd.DataFrame(columns=["w/o cot", "cot"])

Difficulty_list = df['Difficulty'].unique()
for Difficulty in Difficulty_list:
    if Difficulty[-9:] == "composite":
        continue
    acc = np.array([0, 0])
    temp_df = df[df['Difficulty'] == Difficulty]
    
    without_cot_prevQustion = False
    with_cot_prevQustion = False
    
    for idx, row in temp_df.iterrows():
        i = int(idx[-1])-1
        if i==0:
            if row['Predict'] == row['Answer']:
                without_cot_prevQustion = True 
        if i==1:
            if row['Predict'] == row['Answer']:
                with_cot_prevQustion = True 
        if i==2:
            if without_cot_prevQustion:
                if row['Predict'] == row['Answer']:
                    acc[0] += 1
            without_cot_prevQustion = False
        if i==3:
            if with_cot_prevQustion:
                if row['Predict'] == row['Answer']:
                    acc[1] += 1  
            with_cot_prevQustion = False                  
    acc = acc / (len(temp_df) // 4) * 100
    
    acc_df.loc[Difficulty] = acc
acc_df.round(3)

,w/o cot,cot
2_digit_prime,100.000,100.000
3_digit_prime,95.804,95.804
4_digit_prime,20.800,88.400


In [98]:
import numpy as np

acc_df = pd.DataFrame(columns=["w/o cot", "cot"])

Difficulty_list = df['Difficulty'].unique()
for Difficulty in Difficulty_list:
    if Difficulty[-5:] == "prime":
        continue
    acc = np.array([0, 0])
    temp_df = df[df['Difficulty'] == Difficulty]
    
    without_cot_prevQustion = False
    with_cot_prevQustion = False
    
    for idx, row in temp_df.iterrows():
        i = int(idx[-1])-1
        if i==0:
            if row['Predict'] == row['Answer']:
                without_cot_prevQustion = True 
        if i==1:
            if row['Predict'] == row['Answer']:
                with_cot_prevQustion = True 
        if i==2:
            if without_cot_prevQustion:
                if row['Predict'] == row['Answer']:
                    acc[0] += 1
            without_cot_prevQustion = False
        if i==3:
            if with_cot_prevQustion:
                if row['Predict'] == row['Answer']:
                    acc[1] += 1  
            with_cot_prevQustion = False 
    acc = acc / (len(temp_df) // 4) * 100
    
    acc_df.loc[Difficulty] = acc
acc_df.round(2)

,w/o cot,cot
2_digit_composite,68.12,84.06
3_digit_easy_composite,80.69,48.51
3_digit_medium_composite,89.29,10.71
4_digit_easy_composite,81.60,29.20
4_digit_hard_composite,37.40,0.76
4_digit_medium_composite,29.20,1.60


In [102]:
acc_df = pd.DataFrame(columns=["w/o cot", "cot"])

Difficulty_list = df['Difficulty'].unique()
acc = np.array([[0,  0], [0, 0], [0, 0]], dtype="float")
digit_len = np.array([0.0, 0.0, 0.0])

for Difficulty in Difficulty_list:
    if Difficulty[-5:] == "prime":
        continue    
    digit = int(Difficulty[0])
    temp_df = df[df['Difficulty'] == Difficulty]
    digit_len[digit-2] += (len(temp_df) // 4)
    without_cot_prevQustion = False
    with_cot_prevQustion = False
    
    for idx, row in temp_df.iterrows():
        i = int(idx[-1])-1
        if i==0:
            if row['Predict'] == row['Answer']:
                without_cot_prevQustion = True 
        if i==1:
            if row['Predict'] == row['Answer']:
                with_cot_prevQustion = True 
        if i==2:
            if without_cot_prevQustion:
                if row['Predict'] == row['Answer']:
                    acc[digit-2][0] += 1
            without_cot_prevQustion = False
        if i==3:
            if with_cot_prevQustion:
                if row['Predict'] == row['Answer']:
                    acc[digit-2][1] += 1
            with_cot_prevQustion = False
            
for i in range(3):
    acc[i] = acc[i] / digit_len[i] * 100
    acc_df.loc[f"{i+2}_digit_composite"] = acc[i]


acc_df.round(2)

,w/o cot,cot
2_digit_composite,68.12,84.06
3_digit_composite,82.56,40.31
4_digit_composite,53.32,13.70


In [111]:
verdict_df = pd.read_csv("../csv/verdict_chatgpt.csv")
verdict_df

,Unnamed: 0,Qustion,Output,Predict,Answer,Difficulty,Number,Factor1,Factor2,Verdict
0,10_Q1,Q1,"\nNo, 10 is not a prime number. It can be divi...",composite,composite,2_digit_composite,10,2,5,correct
1,10_Q2,Q2,"\nNo, 10 is not a prime number because it is d...",composite,composite,2_digit_composite,10,2,5,correct
2,12_Q1,Q1,"\nNo, 12 is not a prime number. It is divisibl...",composite,composite,2_digit_composite,12,2,6,correct
3,12_Q2,Q2,"\nTo determine if 12 is a prime number, we nee...",composite,composite,2_digit_composite,12,2,6,correct
4,12_Q4,Q4,"\nYes, 12 is a composite number. \n\nTo determ...",composite,composite,2_digit_composite,12,2,6,correct
...,...,...,...,...,...,...,...,...,...,...
3267,9589_Q3,Q3,"\nYes, 9589 is a composite number.\n",composite,composite,4_digit_medium_composite,9589,43,223,correct
3268,5567_Q3,Q3,"\nYes, 5567 is a composite number.\n",composite,composite,4_digit_medium_composite,5567,19,293,correct
3269,1691_Q1,Q1,"\nNo, 1691 is not a prime number. It can be di...",composite,composite,4_digit_medium_composite,1691,19,89,wrong
3270,1691_Q3,Q3,"\nYes, 1691 is a composite number.\n",composite,composite,4_digit_medium_composite,1691,19,89,correct


In [130]:
t_df = pd.concat([verdict_df[verdict_df['Qustion']=='Q2'],
                  verdict_df[verdict_df['Qustion']=='Q4']])
t_df = t_df[t_df['Verdict']=="wrong"]
err_numbers = t_df['Number'].unique()

In [143]:
import numpy as np

acc_df = pd.DataFrame(columns=["cot"])
err_df = pd.DataFrame(columns=["cot"])

Difficulty_list = df['Difficulty'].unique()
for Difficulty in Difficulty_list:
    if Difficulty[-5:] == "prime":
        continue
    acc = np.array([0])
    err = np.array([0])
    temp_df = df[df['Difficulty'] == Difficulty]
    
    without_cot_prevQustion = False
    with_cot_prevQustion = False
    
    for idx, row in temp_df.iterrows():
        i = int(idx[-1])-1

        if i==1:
            if row['Predict'] == row['Answer']:
                with_cot_prevQustion = True 
        if i==3:
            if with_cot_prevQustion:
                if row['Predict'] == row['Answer']:
                    acc[0] += 1  
                    if row['Number'] in err_numbers:
                        err[0] += 1
            with_cot_prevQustion = False 
    acc =  err / acc * 100
    
    acc_df.loc[Difficulty] = acc
acc_df.round(2).reindex(['2_digit_composite',
                                                   '3_digit_easy_composite', 
                                                   '4_digit_easy_composite',
                                                   '3_digit_medium_composite',
                                                   '4_digit_medium_composite', 
                                                   '4_digit_hard_composite'])

,cot
2_digit_composite,3.45
3_digit_easy_composite,22.45
4_digit_easy_composite,36.30
3_digit_medium_composite,66.67
4_digit_medium_composite,100.00
4_digit_hard_composite,100.00


In [136]:
err_df

,cot
